In [1]:
import os
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt

In [2]:
kingfa_datasets_path = "./kingfadatasets/"

In [3]:
def wpt1D(data, wavelet='db1', mode='symmetric', maxlevel=2):
    wp = pywt.WaveletPacket(data=data, wavelet=wavelet, mode=mode, maxlevel=maxlevel)
    paths = [node.path for node in wp.get_level(maxlevel)]  # 列出所有树节点路径
    df_wpt = pd.DataFrame()  # 合并所有四级子节点
    for a in paths:
        df_wpt[a] = wp[a].data
    return df_wpt.values

In [4]:
X = np.array(pd.read_csv(kingfa_datasets_path+"data.csv", index_col=0))

In [5]:
X = np.apply_along_axis(wpt1D, 1, X)

In [6]:
y = np.array(pd.read_csv(kingfa_datasets_path+"target.csv", index_col=0))

In [7]:
print(X.shape)
print(y.shape)

(3000, 1024, 4)
(3000, 1)


In [8]:
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam

Using TensorFlow backend.


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1024)

In [10]:
X_train = X_train.reshape(-1, 1024, 4, 1)
X_test = X_test.reshape(-1, 1024, 4, 1)
y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)

In [11]:
model = Sequential()

In [12]:
model.add(Conv2D(filters=16, kernel_size=(6, 4), input_shape=(1024, 4, 1)))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

In [13]:
model.add(Conv2D(filters=32, kernel_size=(4, 2), padding='same'))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

In [14]:
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('sigmoid'))
model.add(Dense(3))
model.add(Activation('softmax'))

In [15]:
adam = Adam(lr=1e-4)

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1019, 1, 16)       400       
_________________________________________________________________
activation_1 (Activation)    (None, 1019, 1, 16)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 510, 1, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 510, 1, 32)        4128      
_________________________________________________________________
activation_2 (Activation)    (None, 510, 1, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 255, 1, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8160)              0         
__________

In [17]:
model.compile(optimizer=adam, loss='squared_hinge', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=300, validation_split=0.1)

Train on 2295 samples, validate on 255 samples
Epoch 1/300
2295/2295 [==============================] - 4s 2ms/step - loss: 0.8211 - acc: 0.3368 - val_loss: 0.8193 - val_acc: 0.3529
Epoch 2/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.8173 - acc: 0.3246 - val_loss: 0.8177 - val_acc: 0.3255
Epoch 3/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.8162 - acc: 0.3425 - val_loss: 0.8148 - val_acc: 0.3255
Epoch 4/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.8148 - acc: 0.3303 - val_loss: 0.8150 - val_acc: 0.3216
Epoch 5/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.8148 - acc: 0.3255 - val_loss: 0.8146 - val_acc: 0.3255
Epoch 6/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.8144 - acc: 0.3307 - val_loss: 0.8143 - val_acc: 0.3216
Epoch 7/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.8150 - acc: 0.3464 - val_loss: 0.8155 - val_acc: 0.3216
Epoch 8/

Epoch 61/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6948 - acc: 0.9015 - val_loss: 0.6948 - val_acc: 0.9137
Epoch 62/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6944 - acc: 0.9020 - val_loss: 0.6944 - val_acc: 0.9176
Epoch 63/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6934 - acc: 0.9050 - val_loss: 0.6946 - val_acc: 0.9098
Epoch 64/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6932 - acc: 0.9098 - val_loss: 0.6934 - val_acc: 0.8941
Epoch 65/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6916 - acc: 0.9255 - val_loss: 0.6946 - val_acc: 0.8824
Epoch 66/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6911 - acc: 0.9285 - val_loss: 0.6947 - val_acc: 0.8824
Epoch 67/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6908 - acc: 0.9259 - val_loss: 0.6912 - val_acc: 0.9294
Epoch 68/300
2295/2295 [========================

2295/2295 [==============================] - 3s 1ms/step - loss: 0.6771 - acc: 0.9664 - val_loss: 0.6817 - val_acc: 0.9412
Epoch 122/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6767 - acc: 0.9673 - val_loss: 0.6784 - val_acc: 0.9490
Epoch 123/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6758 - acc: 0.9699 - val_loss: 0.6770 - val_acc: 0.9608
Epoch 124/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6757 - acc: 0.9669 - val_loss: 0.6770 - val_acc: 0.9686
Epoch 125/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6759 - acc: 0.9686 - val_loss: 0.6784 - val_acc: 0.9490
Epoch 126/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6761 - acc: 0.9686 - val_loss: 0.6796 - val_acc: 0.9412
Epoch 127/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6763 - acc: 0.9708 - val_loss: 0.6841 - val_acc: 0.9333
Epoch 128/300
2295/2295 [==============================

Epoch 181/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6707 - acc: 0.9891 - val_loss: 0.6716 - val_acc: 0.9882
Epoch 182/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6719 - acc: 0.9856 - val_loss: 0.6717 - val_acc: 0.9765
Epoch 183/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6707 - acc: 0.9865 - val_loss: 0.6724 - val_acc: 0.9765
Epoch 184/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6710 - acc: 0.9874 - val_loss: 0.6727 - val_acc: 0.9765
Epoch 185/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6712 - acc: 0.9874 - val_loss: 0.6713 - val_acc: 0.9922
Epoch 186/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6706 - acc: 0.9882 - val_loss: 0.6721 - val_acc: 0.9765
Epoch 187/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6703 - acc: 0.9895 - val_loss: 0.6716 - val_acc: 0.9804
Epoch 188/300
2295/2295 [================

Epoch 241/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6685 - acc: 0.9961 - val_loss: 0.6691 - val_acc: 0.9961
Epoch 242/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6682 - acc: 0.9983 - val_loss: 0.6704 - val_acc: 0.9843
Epoch 243/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6681 - acc: 0.9978 - val_loss: 0.6691 - val_acc: 0.9961
Epoch 244/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6680 - acc: 0.9987 - val_loss: 0.6696 - val_acc: 0.9922
Epoch 245/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6684 - acc: 0.9965 - val_loss: 0.6690 - val_acc: 0.9922
Epoch 246/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6682 - acc: 0.9965 - val_loss: 0.6688 - val_acc: 0.9961
Epoch 247/300
2295/2295 [==============================] - 3s 1ms/step - loss: 0.6686 - acc: 0.9961 - val_loss: 0.6709 - val_acc: 0.9804
Epoch 248/300
2295/2295 [================

In [ ]:
score = model.evaluate(X_test, y_test)
print("LOSS:", score[0])
print("Accuracy:", score[1])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()